# Assignment 7: Expectation Maximization

MIDS W281: Computer Vision

## Recommended Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from skimage.color import rgb2gray
import skimage.feature
from matplotlib.patches import Circle

![Hybrid Teaser](https://raw.githubusercontent.com/W281/fileRepository/main/Assignments/output.gif)

 ### Overview
In tasks with noisy input data, it is often necessary to estimate parameters using an iterative approach. To illustrate this, we have provided you a realistic (but very simple) optimization problem. 

You are tasked with writing the software for a parts inspection system. The input to this system is a color image of a circular part. Your task is to locate and fit a circle (center and radius) to the part, and then determine whether the size of the part (its radius) is within a specified tolerance. You will use the expectation/maximization (EM) algorithm to simultaneously locate the boundary of the circular part, fit a circle to the part, and estimate the goodness of the fit.

### Description

1. To begin, your system should load a color (RGB) image, convert the image to grayscale, and extract the salient edges in the image. You can use the Canny edge detector from Python's OpenCV library to find the edges. This function will return a binary image with a value of 1 at a pixel location corresponding to a salient edge and 0 otherwise. Note that you will need to play around with the size of the filter to get a useful output. You might need to exclude the edges of the image and then extract the x- and y-coordinates of all pixels with an edge value equal to 1 (see numpy's `argwhere`). It is from these coordinates that we will find and fit a circle.  

2. The coordinates of each edge pixel correspond to one of two models:   
    (1) a circle with center $(c_x,c_y)$ and radius $(r)$; or  
    (2) an "outlier" model.  
    The probability of belonging to model 1 will be modeled as a Gaussian distribution $(g(d) = exp(-d^2/σ)$ and will depend on the distance, $d$, of each point to the circle. The probability of belonging to model 2 will be modeled as a uniform distribution and fixed to be 1/10.  
    **HINT:** Think about which kind of probability was provided for models 1 and 2, and how it is used throughout the entire EM process.

3. You should begin your EM iterations by initializing $(c_x,c_y)$ to be the image center, $r=190$ pixels, and $σ=128$.   

4. In the E-step, the residual error for model 1 is based on the shortest distance, $d$, between a point $(x,y)$ and the current estimate of the circle with center $(c_x,c_y)$ and radius, $r$. This distance is:  
    $$d=\left\lvert \sqrt{ (x-c_x)^2 + (y-c_y)^2} - r \right\rvert$$
    
5. In the M-step, you will use weighted total least-squares to re-estimate the paramters of the circle. First, implement the total least-squares estimator described in Section 2 of this [paper](gander_golub_strebel.pdf). The weights of weighted total least-squares estimator are computed in the E-step. Incorporate this weighting into your total least-squares estimator. 

6. After performing the E- and M-step, update the value of $σ$.

7. Repeatedly perform the E- and M-steps until the difference between subsequent estimates of the circle center and radius are each less than 0.1 pixels.  

8. As shown in the sample output above, on each EM iteration you should display two images:  
    (1) the current circle superimposed on top of the original RGB image;  
    (2) the current probability of each pixel location belonging to model 1 visualized as an image. 
    
9. Show your final fitting results on the five images provided in the "discs" directory.  
**If your solution is similar to ours, it might fail on the image disc4.jpg. If so, fix this failure by choosing a better initial value for $(c_x, c_y)$.**   

### Deliverables:

- Python code to perform the above EM algorithm.  
- Plots showing the steps for for EM for each of the image.  
- Plot showing the final fitting results, i.e. the circle super-imposed on the RGB disc image.

### Create your Function to execute the above steps

In [ ]:
def find_EM(fileName):
    # write a function that takes the the filename of an image,
    # converts the image to grayscale,
    # calculates the edges using the Canny edge detector,
    # runs EM process and plots/displays 
    # each step of the process.
    # function should NOT return anything.

    # Load the image
    rgb_im = plt.imread(fileName)

    # convert to grayscale
    ## TODO im = ??? (convert to grayscale)

    # Find edge cordinate

    ## TODO use skimage.feature.canny to find the edges
    ## NOTE: experiment with the parameters to find a reasonable amount of points (<10k)
    ## TODO edgeIm = ???

    ## find the x-y cordinates with edge == 1 **hint: use np.argwhere
    # TODO cord = ???

    ## create X and Y arrays for the coordinates
    Y = cord[:, 0]
    X = cord[:, 1]

    # Print the number of points
    print(f'Number of Points: {noOfPoints}')

    # display the image and plot the edge coordinates
    ## display the image 
    plt.imshow(im) 
    plt.plot(X, Y, 'r.')
    plt.show()

    # Expectation/Maximization
    ## intialize new center (cx, cy)/2 and radius to 190
    new_cx = im.shape[1] / 2.0 # 210
    new_cy = im.shape[0] / 2.0 # 320
    new_r = 190

    prev_cx = 0
    prev_cy = 0
    prev_r = 0

    ## initial sigma
    sigma = 128

    # create empty arrays for expectation and d that correspond to the number of coordinates previously calculated 
    expectation = np.zeros((noOfPoints,))
    d = np.zeros((noOfPoints,))

    # repeat optimization till the difference is greater than 0.1
        ## expectation step
        ### compute the expectation of each point given prev parameters
        ### after each step, display the images with circle and expectation weights 
    
    while((np.abs(prev_cx - new_cx) > 0.1) or (np.abs(prev_cy - new_cy) > 0.1) or (np.abs(prev_r - new_r) > 0.1)):
        
        prev_cx = new_cx
        prev_cy = new_cy
        prev_r = new_r

        ## expectation step
        ### compute the expectation of each point given prev parameters
        for i in range(noOfPoints):

            #TODO d[i] = ??
            #TODO expectation[i] = ??
            
        exIm = np.zeros((im.shape[0], im.shape[1]))

        #for i in range(len(cord)):
        for i in range(noOfPoints):
            r,c = np.unravel_index(cord[i], (im.shape[0], im.shape[1]))[1]
            exIm[r,c] = expectation[i]

        ## display the images with circle and expectation weights
        fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(10,5))
        ax[0].imshow(rgb_im)
        
        circ = Circle( (prev_cx, prev_cy), radius=prev_r, fill = False )
        ax[0].add_patch(circ)
        ax[1].imshow(exIm)
        ax[0].set_title(f'r = {np.around(prev_r, 2)}, cx = {np.around(prev_cx, 2)}, cy = {np.around(prev_cy, 2)}')
        plt.show()

        # maximization step
        # ||WBu|| minimization
        #TODO W = ??
        #TODO B = ??

        # use SVD to solve for u in ||WBu||
        # use the vector with minimum eigen value
                
        # update center and radius
        #TODO new_cx = ??
        #TODO new_cy = ??
        #TODO new_r = ??
        
        # update sigma
        #TODO sigma = ??


### Run your function for all disc images

In [ ]:
for image in os.listdir('discs'):
    print(f'Image: {image}')
    find_EM('discs/' + image)

#### Animation that displays the process

In [ ]:
from IPython.display import Video
Video("images/SampleOutput.mov")